In [93]:
## import json
import rasterio
import rasterio.features
import numpy as np
from tqdm.notebook import tqdm

# read in data and windows
src = rasterio.open('./2020_pop.tif')
print(src.profile)
print(src.bounds)
windows = [window for ij, window in src.block_windows(1)]
np.shape(windows)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': -3.4028234663852886e+38, 'width': 43200, 'height': 18720, 'count': 1, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0083333333, 0.0, -180.001249265,
       0.0, -0.0083333333, 83.99958319871001), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'band'}
BoundingBox(left=-180.001249265, bottom=-72.00041617728999, right=179.99874929500004, top=83.99958319871001)


(12506,)

In [ ]:
window=windows[6000]
data = src.read(1, window=window)
print(src.crs)
print(np.shape(data))
mask = src.dataset_mask(window=window)
print(mask.shape)
mask

In [ ]:
import matplotlib.pyplot as plt
data # why are these not pop numbers
# plt.rcParams['figure.figsize'] = 14, 14
# data[data<0] = -1
plt.imshow(data, )

In [134]:
import json
i=0
collection = {
        'type': 'FeatureCollection', 
        'features': [] }
for geom, val in rasterio.features.shapes(data, transform=src.transform):
    print('BEFOREgeom={},val={}'.format(geom, val))
#     is the same.
    geom = rasterio.warp.transform_geom(src.crs, 'EPSG:4326', geom, precision=5)
    print('geom={},val={}'.format(geom, val))
    i = i + 1
    if i == 100:
        break
    collection['features'].append({
        'type': 'Feature', 
        'properties': {'raster_val': val}, 
        'geometry': geom });

with open('some_windows.json', 'w') as dst:
        json.dump(collection, dst)


BEFOREgeom={'type': 'Polygon', 'coordinates': [[(-180.001249265, 83.99958319871001), (-180.001249265, 83.99124986541001), (-179.99291593170003, 83.99124986541001), (-179.99291593170003, 83.99958319871001), (-180.001249265, 83.99958319871001)]]},val=4.139383792877197
geom={'type': 'Polygon', 'coordinates': [[[-180.00125, 83.99958], [-180.00125, 83.99125], [-179.99292, 83.99125], [-179.99292, 83.99958], [-180.00125, 83.99958]]]},val=4.139383792877197
BEFOREgeom={'type': 'Polygon', 'coordinates': [[(-179.99291593170003, 83.99958319871001), (-179.99291593170003, 83.99124986541001), (-179.9845825984, 83.99124986541001), (-179.9845825984, 83.99958319871001), (-179.99291593170003, 83.99958319871001)]]},val=4.321421146392822
geom={'type': 'Polygon', 'coordinates': [[[-179.99292, 83.99958], [-179.99292, 83.99125], [-179.98458, 83.99125], [-179.98458, 83.99958], [-179.99292, 83.99958]]]},val=4.321421146392822
BEFOREgeom={'type': 'Polygon', 'coordinates': [[(-179.9845825984, 83.99958319871001), (

In [29]:
def readwindow(src, window):
    data = src.read(window=window)
    shapes = rasterio.features.shapes(data)
    results = []
    for (s, v) in shapes:
        if v < 0 or v < 0.5:
            continue
        
        results.append({
        'type': 'Feature', 
        'properties': {'raster_val': v}, 
        'geometry': s })
    
    return list(results)

def readwindows(src):
    windows = [window for ij, window in src.block_windows()]
    results = []
    print(np.shape(windows))
    for i in tqdm(range(1000)):
        result = readwindow(src, windows[i])
        results = np.concatenate([results, result])
        
    collection = {
        'type': 'FeatureCollection', 
        'features': list(results) }
    print(np.size(list(results)))
    return collection


In [3]:
with rasterio.open('./2020_pop.tif') as src:
    print(src.name)
    print(src.count)
    print(src.width)
    print(src.height)
    print(src.indexes)
    print(src.dtypes)
    print(src.bounds)
    print(src.block_shapes)
    
    collection = readwindows(src)
    
    with open('some_windows.json', 'w') as dst:
        json.dump(collection, dst)

./2020_pop.tif
1
43200
18720
(1,)
('float32',)
BoundingBox(left=-180.001249265, bottom=-72.00041617728999, right=179.99874929500004, top=83.99958319871001)
[(256, 256)]


NameError: name 'readwindows' is not defined